In [ ]:
!pip -q install nlpaug
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
from random import shuffle
from tqdm import tqdm
import numpy as np

In [ ]:
text = "weather is wonderful today. Should we go outside?"
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
augmented_text = aug.augment(text)
augmented_text

['your weather is wonderful today. should we maybe go hide outside?']

In [ ]:
dataset = pd.read_excel("/content/drive/MyDrive/ProjectDL/test_english.xlsx")

In [ ]:
dataset

,prompt,completion
0,Hello,Hello
1,how are you,let's say good get well how are you
2,Thank you I'm fine too,God bless you I'm glad
3,Where are you from,"My family is from Konya, but I was born and ra..."
4,"Oh how beautiful, what is Konya famous for?",It has quite a famous thing though. I can say ...
...,...,...
94,What is AIHAN?,AIHAN is an artificial intelligence model deve...
95,Where is Alihan Sağöz?,Alihan Sağöz is currently offline and I will t...
96,who are you?,My name is AIHAN. I am an artificial intellige...
97,what's your name?,my name is AIHAN.


In [ ]:
def augment_text(df_n,samples=300,pr=0.2):
    aug.aug_p=pr
    new_text=[]
    new_target=[]
    
    ## data augmentation loop
    print(len(df_n))
    for i in tqdm(np.random.randint(0,len(df_n),samples)):
            text = df_n.iloc[i]['prompt']
            augmented_text = aug.augment(text)
            new_text.append(augmented_text[0])
            text = df_n.iloc[i]['completion']
            augmented_text = aug.augment(text)
            new_target.append(augmented_text[0])

    ## dataframe
    new=pd.DataFrame({'prompt':new_text,'completion':new_target})
    df_n= pd.concat([df_n, new]).sample(frac=1).reset_index(drop=True)
    return df_n

In [ ]:
dataset2 = augment_text(dataset)

99


100%|██████████| 300/300 [06:08<00:00,  1.23s/it]


In [ ]:
dataset2['prompt'] =  dataset2['prompt'].astype(str) + '\n'
dataset2['completion'] =  dataset2['completion'].astype(str) + '\n'
dataset2

,prompt,completion
0,what do it i do need to do to contact aihan\n,you just send a message on facebook whatsapp. ...
1,if you already had had the earliest chance to ...,i personally would kill all the known terroris...
2,so what's with your name?\n,my birth name is master aihan.\n
3,What do I need to do to contact Aihan\n,Just send a message on Whatsapp. If Alihan is ...
4,then if you really could be a living celebrity...,first cem died yilmaz.\n
...,...,...
394,Which movie star do you like the most?\n,Tom Hardy and Jason Mamoa because I love their...
395,what planet was the most beautiful special gif...,and the diamond bracelet my girlfriend bought.\n
396,if then you ever had a chance to completely ch...,now i would stop the wars first generation net.\n
397,"as i understand, and thank you constantly for ...",but you're welcome always. gage my new favorit...


In [ ]:
cols = ['completion', 'prompt']
dataset2 = dataset2[cols]
dataset2

,completion,prompt
0,you just send a message on facebook whatsapp. ...,what do it i do need to do to contact aihan\n
1,i personally would kill all the known terroris...,if you already had had the earliest chance to ...
2,my birth name is master aihan.\n,so what's with your name?\n
3,Just send a message on Whatsapp. If Alihan is ...,What do I need to do to contact Aihan\n
4,first cem died yilmaz.\n,then if you really could be a living celebrity...
...,...,...
394,Tom Hardy and Jason Mamoa because I love their...,Which movie star do you like the most?\n
395,and the diamond bracelet my girlfriend bought.\n,what planet was the most beautiful special gif...
396,now i would stop the wars first generation net.\n,if then you ever had a chance to completely ch...
397,but you're welcome always. gage my new favorit...,"as i understand, and thank you constantly for ..."


In [ ]:
dataset2.to_csv(r'/content/drive/MyDrive/ProjectDL/dataset_en_aug.csv')